# Инициализация

Загружаем библиотеки необходимые для выполнения кода ноутбука.

In [1]:
import pandas as pd
import numpy as np
import subprocess
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

RANDOM_STATE = 42

# === ЭТАП 1 ===

# Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

In [3]:
def load_parquet_file(filename, base_url="https://storage.yandexcloud.net/mle-data/ym/"):
    """
    Универсальная функция для загрузки parquet файлов.
    
    Args:
        filename (str): Имя файла для загрузки (например, "tracks.parquet")
        base_url (str): Базовый URL для скачивания файлов
    
    Returns:
        pd.DataFrame или None: Загруженные данные или None при ошибке
    """
    # Формируем полный URL
    file_url = f"{base_url}{filename}"
    
    try:
        data = pd.read_parquet(filename)
        print(f"✓ {filename} загружен локально")
        return data
        
    except FileNotFoundError:
        print(f"{filename} не найден, скачиваю из {file_url}...")
        try:
            subprocess.run(['wget', file_url], check=True)
            data = pd.read_parquet(filename)
            print(f"✓ {filename} успешно скачан и загружен")
            return data
        except subprocess.CalledProcessError:
            print(f"❌ Ошибка при скачивании {filename}")
            return None
        except Exception as e:
            print(f"❌ Ошибка при чтении скачанного {filename}: {e}")
            return None
    except Exception as e:
        print(f"❌ Ошибка при чтении {filename}: {e}")
        return None

In [4]:
# Использование функции
tracks = load_parquet_file("tracks.parquet")
catalog_names = load_parquet_file("catalog_names.parquet")
interactions = load_parquet_file("interactions.parquet")

✓ tracks.parquet загружен локально
✓ catalog_names.parquet загружен локально
✓ interactions.parquet загружен локально


In [5]:
# Проверяем загрузку
if all(x is not None for x in [tracks, catalog_names, interactions]):
    print("✓ Все файлы успешно загружены!")
    print(f"Tracks: {tracks.shape}")
    print(f"Catalog names: {catalog_names.shape}")
    print(f"Interactions: {interactions.shape}")
else:
    print("⚠ Не все файлы удалось загрузить")

✓ Все файлы успешно загружены!
Tracks: (1000000, 4)
Catalog names: (1812471, 3)
Interactions: (222629898, 4)


# Обзор данных

tracks.parquet - данные о треках:  
 - track_id — идентификатор музыкального трека;  
 - albums —  список идентификаторов альбомов, содержащих трек;  
 - artists — список идентификаторов исполнителей трека;  
 - genres — список идентификаторов жанров, к которым принадлежит трек.

In [6]:
tracks.head()

,track_id,albums,artists,genres
0,26,"[3, 2490753]",[16],"[11, 21]"
1,38,"[3, 2490753]",[16],"[11, 21]"
2,135,"[12, 214, 2490809]",[84],[11]
3,136,"[12, 214, 2490809]",[84],[11]
4,138,"[12, 214, 322, 72275, 72292, 91199, 213505, 24...",[84],[11]


In [7]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   track_id  1000000 non-null  int64 
 1   albums    1000000 non-null  object
 2   artists   1000000 non-null  object
 3   genres    1000000 non-null  object
dtypes: int64(1), object(3)
memory usage: 30.5+ MB


In [8]:
is_unique = tracks['track_id'].nunique() == len(tracks)
unique_count = tracks['track_id'].nunique()
total_count = len(tracks)

print(f"📊 Уникальность значений:")
print(f"   • Всего записей: {total_count:,}")
print(f"   • Уникальных track_id: {unique_count:,}")
print(f"   • Все track_id уникальны: {'✅ ДА' if is_unique else '❌ НЕТ'}")
if not is_unique:
    duplicates = total_count - unique_count
    print(f"   • Дубликатов найдено: {duplicates:,}")

📊 Уникальность значений:
   • Всего записей: 1,000,000
   • Уникальных track_id: 1,000,000
   • Все track_id уникальны: ✅ ДА


In [9]:
tracks[tracks["albums"].str.len() == 0]

,track_id,albums,artists,genres
310821,20200372,[],[],[]
310826,20200380,[],[],[]
312469,20305116,[],[],[]
312474,20305121,[],[],[]
320353,20756854,[],[],[]
326588,21196099,[],[],[]
326592,21196103,[],[],[]
326594,21196105,[],[],[]
326596,21196107,[],[],[]
326598,21196109,[],[],[]


есть небольшое кол-во пустых списков идентификаторов альбомов (с пустыми для них идентификаторами исполнителей трека и жанров)

In [10]:
def check_empty_lists(df, column_name, entity_name=None):
    """
    Проверяет колонку с списками на пустые значения
    
    Parameters:
    df - DataFrame для анализа
    column_name - название колонки со списками
    entity_name - человекочитаемое название сущности (опционально)
    """
    
    # Подсчитываем пустые списки
    empty_mask = df[column_name].str.len() == 0
    count_empty = empty_mask.sum()
    total_count = len(df)
    
    print(f"Записей с пустыми {entity_name}: {count_empty:,}")
    print(f"Процент от общего числа: {count_empty/total_count*100:.4f}%")
    print(f"Всего записей: {total_count:,}")
    print(f"Записей с {entity_name}: {total_count - count_empty:,}")
    
    if count_empty > 0:
        # Получаем примеры ID с пустыми списками
        if 'track_id' in df.columns:
            sample_ids = df.loc[empty_mask, 'track_id'].head(5).tolist()
            print(f"\n📋 Примеры track_id с пустыми {entity_name}:")
            for track_id in sample_ids:
                print(f"   • {track_id}")
        else:
            # Если нет track_id, показываем индексы
            sample_indices = df[empty_mask].index[:5].tolist()
            print(f"\n📋 Примеры индексов с пустыми {entity_name}:")
            for idx in sample_indices:
                print(f"   • Индекс: {idx}")
    
    print("=" * 50)
    return empty_mask

In [11]:
albums_result = check_empty_lists(tracks, 'albums', 'альбомами')

Записей с пустыми альбомами: 18
Процент от общего числа: 0.0018%
Всего записей: 1,000,000
Записей с альбомами: 999,982

📋 Примеры track_id с пустыми альбомами:
   • 20200372
   • 20200380
   • 20305116
   • 20305121
   • 20756854


In [12]:
# Удаляем треки без альбомов
initial_count = len(tracks)
track_id_without_albums = tracks[tracks["albums"].str.len() == 0]["track_id"]
tracks = tracks[~tracks["track_id"].isin(track_id_without_albums)]
final_count = len(tracks)

print(f"Удалено треков: {initial_count - final_count:,}")
print(f"Осталось треков: {final_count:,}")
print("=" * 45)

Удалено треков: 18
Осталось треков: 999,982


In [13]:
artists_result = check_empty_lists(tracks, 'artists', 'артистами')

Записей с пустыми артистами: 15,351
Процент от общего числа: 1.5351%
Всего записей: 999,982
Записей с артистами: 984,631

📋 Примеры track_id с пустыми артистами:
   • 3599314
   • 3599591
   • 4790215
   • 10063296
   • 12122918


In [14]:
# Удаляем треки без артистов
initial_count = len(tracks)
track_id_without_artists = tracks[tracks["artists"].str.len() == 0]["track_id"]
tracks = tracks[~tracks["track_id"].isin(track_id_without_artists)]
final_count = len(tracks)

print(f"Удалено треков: {initial_count - final_count:,}")
print(f"Осталось треков: {final_count:,}")
print("=" * 45)

Удалено треков: 15,351
Осталось треков: 984,631


In [15]:
genres_result = check_empty_lists(tracks, 'genres', 'жанрами')

Записей с пустыми жанрами: 3,654
Процент от общего числа: 0.3711%
Всего записей: 984,631
Записей с жанрами: 980,977

📋 Примеры track_id с пустыми жанрами:
   • 2520
   • 16776
   • 16801
   • 23752
   • 38012


In [16]:
# Удаляем треки без жанров
initial_count = len(tracks)
track_id_without_genres = tracks[tracks["genres"].str.len() == 0]["track_id"]
tracks = tracks[~tracks["track_id"].isin(track_id_without_genres)]
final_count = len(tracks)

print(f"Удалено треков: {initial_count - final_count:,}")
print(f"Осталось треков: {final_count:,}")
print("=" * 45)

Удалено треков: 3,654
Осталось треков: 980,977


In [17]:
# Объединяем все списки удаленных треков
track_id_deleted = track_id_without_albums.tolist() + track_id_without_artists.tolist() + track_id_without_genres.tolist()

print("📊 СТАТИСТИКА УДАЛЕННЫХ ТРЕКОВ")
print("=" * 40)

# Подсчитываем общую статистику
total_deleted = len(track_id_deleted)
unique_deleted = len(set(track_id_deleted))

print(f"Всего удалено записей: {total_deleted:,}")
print(f"Уникальных track_id: {unique_deleted:,}")

# Статистика по исходным данным
#initial_count = 1000000  # Замените на ваше исходное количество
remaining_count = unique_count - unique_deleted

print(f"\n📈 ОБЩАЯ СТАТИСТИКА:")
print(f"Исходное количество треков: {unique_count:,}")
print(f"Удалено уникальных треков: {unique_deleted:,}")
print(f"Осталось треков: {remaining_count:,}")
print(f"Процент удаленных: {unique_deleted/initial_count*100:.2f}%")

📊 СТАТИСТИКА УДАЛЕННЫХ ТРЕКОВ
Всего удалено записей: 19,023
Уникальных track_id: 19,023

📈 ОБЩАЯ СТАТИСТИКА:
Исходное количество треков: 1,000,000
Удалено уникальных треков: 19,023
Осталось треков: 980,977
Процент удаленных: 1.93%


In [18]:
# Сохраняем список в CSV
deleted_df = pd.DataFrame({'deleted_track_id': track_id_deleted})
deleted_df.to_csv('deleted_tracks.csv', index=False)

print(f"💾 Список из {len(track_id_deleted):,} track_id сохранен в 'deleted_tracks.csv'")

💾 Список из 19,023 track_id сохранен в 'deleted_tracks.csv'


In [19]:
# Преобразуем track_id из int64 в int32
tracks['track_id'] = tracks['track_id'].astype('int32')

# Проверим результат
print(f"Новый тип track_id: {tracks['track_id'].dtype}")
print(f"Память после преобразования:")
tracks.info()

Новый тип track_id: int32
Память после преобразования:
<class 'pandas.core.frame.DataFrame'>
Index: 980977 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   track_id  980977 non-null  int32 
 1   albums    980977 non-null  object
 2   artists   980977 non-null  object
 3   genres    980977 non-null  object
dtypes: int32(1), object(3)
memory usage: 33.7+ MB


In [20]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 980977 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   track_id  980977 non-null  int32 
 1   albums    980977 non-null  object
 2   artists   980977 non-null  object
 3   genres    980977 non-null  object
dtypes: int32(1), object(3)
memory usage: 33.7+ MB


_______________________________________________

catalog_names.parquet - имена артистов, названия альбомов, треков и жанров:  
 - id — идентификатор одной из каталожных единиц (трека, альбома, исполнителя, жанра);  
 - type — тип идентификатора;  
 - name — имя (название) каталожной единицы.

In [21]:
catalog_names.head()

,id,type,name
0,3,album,Taller Children
1,12,album,Wild Young Hearts
2,13,album,Lonesome Crow
3,17,album,Graffiti Soul
4,26,album,Blues Six Pack


In [22]:
catalog_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812471 entries, 0 to 1812470
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   type    object
 2   name    object
dtypes: int64(1), object(2)
memory usage: 41.5+ MB


_____________________

interactions.parquet - данные о том, какие пользователи прослушали тот или иной трек:  
- user_id — идентификатор пользователя,  
- track_id — идентификатор музыкального трека,  
- track_seq — номер места трека в истории пользователя,  
- started_at — дата начала прослушивания трека.

In [23]:
interactions.head()

,user_id,track_id,track_seq,started_at
0,0,99262,1,2022-07-17
1,0,589498,2,2022-07-19
2,0,590262,3,2022-07-21
3,0,590303,4,2022-07-22
4,0,590692,5,2022-07-22


In [24]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 222629898 entries, 0 to 291
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   user_id     int32         
 1   track_id    int32         
 2   track_seq   int16         
 3   started_at  datetime64[ns]
dtypes: datetime64[ns](1), int16(1), int32(2)
memory usage: 5.4 GB


Проверяем данные, есть ли с ними явные проблемы.

In [25]:
interactions_clean = interactions[~interactions['track_id'].isin(track_id_deleted)].reset_index(drop=True)

In [26]:
# Убедимся, что теперь типы совпадают
print(f"Тип track_id в tracks: {tracks['track_id'].dtype}")
print(f"Тип track_id в interactions: {interactions['track_id'].dtype}")

# Проверим, что все track_id из interactions есть в tracks
missing_tracks = set(interactions['track_id']) - set(tracks['track_id'])
print(f"Отсутствующих track_id в tracks: {len(missing_tracks)}")

Тип track_id в tracks: int32
Тип track_id в interactions: int32
Отсутствующих track_id в tracks: 19023


In [27]:
interactions_clean["track_id"].nunique()

980977

In [28]:
len(interactions_clean)

222184449

In [29]:
print(f"Процент оставшихся: {len(interactions_clean)*100/len(interactions):.2f}%")

Процент оставшихся: 99.80%


In [30]:
interactions_clean[interactions_clean["track_seq"].isnull()]

,user_id,track_id,track_seq,started_at


пропусков по track_seq нет

In [31]:
interactions_clean["track_seq"].describe().apply(lambda x: f"{x:0.1f}")

count    222184449.0
mean           462.0
std            824.8
min              1.0
25%             56.0
50%            181.0
75%            505.0
max          16637.0
Name: track_seq, dtype: object

- max(track_seq) = 16637  
- min(track_seq) = 1  
- 75% track_seq меньше 506 
- на будущее: посчитать их за выбросы и удалить из данных

In [32]:
# оставим только взаимодействия с номерами места трека в истории пользователя меньше-равно 1000
interactions_clean = interactions_clean[interactions_clean["track_seq"] <= 1000].reset_index(drop=True)
interactions_clean.head()

,user_id,track_id,track_seq,started_at
0,0,99262,1,2022-07-17
1,0,589498,2,2022-07-19
2,0,590262,3,2022-07-21
3,0,590303,4,2022-07-22
4,0,590692,5,2022-07-22


In [33]:
print(f"Процент оставшихся взаимодействий: {len(interactions_clean)*100/len(interactions):.2f}%")

Процент оставшихся взаимодействий: 87.81%


In [34]:
interactions_clean[interactions_clean["started_at"].isnull()]
# пропусков по started_at нет

,user_id,track_id,track_seq,started_at


In [35]:
interactions_clean["started_at"].describe()

count                        195486833
mean     2022-08-26 18:29:16.756822016
min                2022-01-01 00:00:00
25%                2022-06-27 00:00:00
50%                2022-09-12 00:00:00
75%                2022-11-07 00:00:00
max                2022-12-31 00:00:00
Name: started_at, dtype: object

- min(started_at) = 2022-01-01  
- max(started_at) = 2022-12-31  
- все корректно, данные датированы за весь 2022 год

In [ ]:
# Пример данных по взаимодействиям:
print(interactions_clean.sample(5, random_state=RANDOM_STATE)
                        .sort_values("user_id")
                        .set_index(["user_id", "track_id"])
                        .to_string())

                  track_seq started_at
user_id track_id                      
23030   62107387         62 2022-10-10
398657  38714371        473 2022-11-01
443853  2193302          36 2022-12-06
1339870 45430540        282 2022-08-31
1363369 433550           13 2022-12-23


# Выводы

Приведём выводы по первому знакомству с данными:
- есть ли с данными явные проблемы,
- какие корректирующие действия (в целом) были предприняты.

# === ЭТАП 2 ===

# EDA

Распределение количества прослушанных треков.

Наиболее популярные треки

Наиболее популярные жанры

Треки, которые никто не прослушал

# Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

# Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

# Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

# === ЭТАП 3 ===

# Загрузка данных

Если необходимо, то загружаем items.parquet, events.parquet.

# Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

# Топ популярных

Рассчитаем рекомендации как топ популярных.

# Персональные

Рассчитаем персональные рекомендации.

# Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

# Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

# Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

# Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

# === Выводы, метрики ===

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.